In [1]:
import pandas as pd
from splink.datasets import splink_datasets
from splink.duckdb.linker import DuckDBLinker
from splink.duckdb.blocking_rule_library import block_on
import splink.duckdb.comparison_template_library as ctl
import splink.duckdb.comparison_library as cl

In [4]:
df = splink_datasets.febrl3
df = df.rename(columns=lambda x: x.strip())

df["cluster"] = df["rec_id"].apply(lambda x: "-".join(x.split('-')[:2]))

# dob and ssn needs to be a string for fuzzy comparisons like levenshtein to be applied
df["date_of_birth"] = df["date_of_birth"].astype(str).str.strip()
df["date_of_birth"] = df["date_of_birth"].replace("", None)

df["soc_sec_id"] = df["soc_sec_id"].astype(str).str.strip()
df["soc_sec_id"] = df["soc_sec_id"].replace("", None)

df["postcode"] = df["postcode"].astype(str).str.strip()
df["postcode"] = df["postcode"].replace("", None)

In [5]:
blocking_rules = [
        block_on("soc_sec_id"),
        block_on("given_name"),
        block_on("surname"),
        block_on("date_of_birth"),
        block_on("postcode"),
]

In [6]:
settings = {
    "unique_id_column_name": "rec_id",
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": blocking_rules,
    "comparisons": [
        ctl.name_comparison("given_name", term_frequency_adjustments=True),
        ctl.name_comparison("surname", term_frequency_adjustments=True),
        ctl.date_comparison("date_of_birth", 
                            damerau_levenshtein_thresholds=[],
                            cast_strings_to_date=True,
                            invalid_dates_as_null=True,
                            date_format="%Y%m%d"),
        cl.levenshtein_at_thresholds("soc_sec_id", [2]),
        cl.exact_match("street_number", term_frequency_adjustments=True),
        cl.exact_match("postcode", term_frequency_adjustments=True),
    ],
    "retain_intermediate_calculation_columns": True,
}

linker = DuckDBLinker(df, settings)

In [7]:
deterministic_rules = [
    "l.soc_sec_id = r.soc_sec_id",
    "l.given_name = r.given_name and l.surname = r.surname and l.date_of_birth = r.date_of_birth",
    "l.given_name = r.surname and l.surname = r.given_name and l.date_of_birth = r.date_of_birth"
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.9)

Probability two random records match is estimated to be  0.000528.
This means that amongst all possible pairwise record comparisons, one in 1,894.84 are expected to match.  With 12,497,500 total possible comparisons, we expect a total of around 6,595.56 matching pairs


In [8]:
linker.estimate_u_using_random_sampling(max_pairs=1e6)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - given_name (no m values are trained).
    - surname (no m values are trained).
    - date_of_birth (no m values are trained).
    - soc_sec_id (no m values are trained).
    - street_number (no m values are trained).
    - postcode (no m values are trained).


In [9]:
em_blocking_rule_1 = block_on("substr(date_of_birth,1,3)")
em_blocking_rule_2 = block_on("substr(postcode,1,2)")
session_dob = linker.estimate_parameters_using_expectation_maximisation(em_blocking_rule_1)
session_postcode = linker.estimate_parameters_using_expectation_maximisation(em_blocking_rule_2)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
SUBSTR(l."date_of_birth", 1, 3) = SUBSTR(r."date_of_birth", 1, 3)

Parameter estimates will be made for the following comparison(s):
    - given_name
    - surname
    - soc_sec_id
    - street_number
    - postcode

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - date_of_birth

Iteration 1: Largest change in params was -0.481 in probability_two_random_records_match
Iteration 2: Largest change in params was 0.00951 in the m_probability of given_name, level `All other comparisons`
Iteration 3: Largest change in params was 0.000534 in the m_probability of given_name, level `All other comparisons`
Iteration 4: Largest change in params was 4.99e-05 in the m_probability of soc_sec_id, level `All other comparisons`

EM converged after 4 iterations

Your model is not yet fully trained. Missing estimates for:
    - date_of_b

In [10]:
linker.save_model_to_json("model_febrl3.json")

{'unique_id_column_name': 'rec_id',
 'link_type': 'dedupe_only',
 'blocking_rules_to_generate_predictions': [{'blocking_rule': 'l."soc_sec_id" = r."soc_sec_id"',
   'sql_dialect': 'duckdb'},
  {'blocking_rule': 'l."given_name" = r."given_name"',
   'sql_dialect': 'duckdb'},
  {'blocking_rule': 'l."surname" = r."surname"', 'sql_dialect': 'duckdb'},
  {'blocking_rule': 'l."date_of_birth" = r."date_of_birth"',
   'sql_dialect': 'duckdb'},
  {'blocking_rule': 'l."postcode" = r."postcode"', 'sql_dialect': 'duckdb'}],
 'comparisons': [{'output_column_name': 'given_name',
   'comparison_levels': [{'sql_condition': '"given_name_l" IS NULL OR "given_name_r" IS NULL',
     'label_for_charts': 'Null',
     'is_null_level': True},
    {'sql_condition': '"given_name_l" = "given_name_r"',
     'label_for_charts': 'Exact match given_name',
     'm_probability': 0.5755394090218955,
     'u_probability': 0.003961986081830378,
     'tf_adjustment_column': 'given_name',
     'tf_adjustment_weight': 1.0},